In [1]:
import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...

@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...

def sum(it, /, start=0):
    return functools.reduce(operator.add, it, start)

In [20]:
from collections.abc import Callable, Iterable
from typing import Protocol, Any, TypeVar, overload, Union

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

T = TypeVar('T')
LT = TypeVar('LT', bound=SupportsLessThan)
DT = TypeVar('DT')
MISSING = object()
EMPTY_MSG = 'max() arg is an empty sequence'

# 1
@overload
def max(__arg1: LT, __arg2: LT, *args: LT, key: None = ...) -> LT:
    ...
# 2
@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT]) -> T:
    ...
# 3
@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT:
    ...
# 4
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT]) -> T:
    ...
# 5
@overload
def max(__iterable: Iterable[LT], *, key: None = ..., default: DT) -> Union[LT, DT]:
    ...
# 6
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT], default: DT) -> Union[T, DT]:
    ...

def max(first, *args, key=None, default=MISSING):
    if args:
        series = args
        candidate = first
    else:
        series = iter(first)
        try:
            candidate = next(series)
        except StopIteration:
            if default is not MISSING:
                return default
            raise ValueError(EMPTY_MSG) from None
    if key is None:
        for current in series:
            if candidate < current:
                candidate = current
    else:
        candidate_key = key(candidate)
        for current in series:
            current_key = key(current)
            if candidate_key < current_key:
                candidate = current
                candidate_key = current_key
    return candidate

In [21]:
# 1 and 3 supports less than but key and default not provided
# 1
max(1,2,-3)

2

In [22]:
# 3
max(['Go', 'Python', 'Rust'])

'Rust'

In [23]:
# 2 and 4 argument key provided but no default
# 2
max(1, 2, -3, key=abs)

-3

In [24]:
# 4
max(['Go', 'Python', 'Rust'], key=len)

'Python'

In [25]:
# 5 argument default provided but no key
max([1,2,-3], default=0)

2

In [26]:
# also 5
max([], default=None)

In [27]:
# 6 arguments key and default provided
max([1,2,-3], key=abs, default=None)

-3

In [29]:
max([], key=abs, default=None)

In [30]:
from typing import TypedDict

class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int

In [31]:
pp = BookDict(title='Programming Pearls', authors='Jon Bentley', isbn='0201657880', pagecount=256)
pp

{'title': 'Programming Pearls',
 'authors': 'Jon Bentley',
 'isbn': '0201657880',
 'pagecount': 256}

In [32]:
type(pp)

dict

In [33]:
pp.title # throws

AttributeError: 'dict' object has no attribute 'title'

In [34]:
pp['title']

'Programming Pearls'

In [35]:
BookDict.__annotations__

{'isbn': str, 'title': str, 'authors': list[str], 'pagecount': int}

In [37]:
from typing import TYPE_CHECKING

def demo_books() -> None:
    book = BookDict(isbn='0134757599', title='Refactoring 2e', authors=['Martin Fowler', 'Kent Beck'], pagecount=478)
    authors = book['authors']
    if TYPE_CHECKING:
        reveal_type(authors)
    authors = 'Bob'
    book['weight'] = 4.2
    del book['title']

if __name__ == '__main__':
    demo_books()

In [38]:
AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'

def to_xml(book: BookDict) -> str:
    elements: list[str] = []
    for key, value in book.items():
        if isinstance(value, list):
            elements.extend(AUTHOR_ELEMENT.format(n) for n in value)
        else:
            tag = key.upper()
            elements.append(f'<{tag}>{value}</{tag}>')
    xml = '\n\t'.join(elements)
    return f'<BOOK>\n\t{xml}\n</BOOK>'

In [39]:
def from_json(data: str) -> BookDict:
    json_data: BookDict = json.loads(data)
    return json_data

In [40]:
# casting can silence mypy errors but don't do it often as it is a code smell

In [41]:
# if you need annotations at runtim avoid using __annotations__ directly, use inspect.get_annoations or typing.get_type_hints

In [48]:
from typing import TypeVar, Generic

class Beverage:
    """Any beverage."""

class Juice(Beverage):
    """Any fruit juice"""

class OrangeJuice(Juice):
    """Delicious juice from Brazilian oranges"""

T = TypeVar('T')

class BeverageDispenser(Generic[T]):
    """A dispenser parameterized on the beverage type."""
    def __init__(self, beverage: T) -> None:
        self.beverage = beverage

    def dispense(self) -> T:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
    """Install a fruit juice dispenser."""
    

In [62]:
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser) # GOOD! MyPy is happy

In [45]:
beverage_dispenser = BeverageDispenser(Beverage())
install(beverage_dispenser) # incompatible type "BeverageDispenser[Beverage]" expected "BeverageDispenser[Juice]"

In [49]:
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser) # incompatible type "BeverageDispenser[OrangeJuice]" expected "BeverageDispenser[Juice]"

In [50]:
# Covariant
T_co = TypeVar('T_co', covariant=True)

class BeverageDispenser(Generic[T_co]):
    def __init__(self, beverage: T_co) -> None:
        self.beverage = beverage

    def dispense(self) -> T_co:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
    """Install a fruit juice dispenser."""

In [60]:
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser) # GOOD! MyPy is happy

In [61]:
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser) # GOOD! MyPy is happy

In [53]:
beverage_dispenser = BeverageDispenser(Beverage())
install(beverage_dispenser) # incompatible type "BeverageDispenser[Beverage]" expected "BeverageDispenser[Juice]"

In [55]:
# All genral trash is called Refuse
from typing import TypeVar, Generic

T_contra = TypeVar('T_contra', contravariant=True)

class Refuse:
    """Any refuse."""

class Biodegradable(Refuse):
    """Biodegradable refuse"""

class Compostable(Biodegradable):
    """Compostable refuse"""

class TrashCan(Generic[T_contra]):
    def put(self, refuse: T_contra) -> None:
        """Store trash until dumped."""

def deploy(trash_can: TrashCan[Biodegradable]):
    """Deploy a trash can for biodegradable refuse"""
        

In [63]:
bio_can: TrashCan[Biodegradable] = TrashCan()
deploy(bio_can) # GOOD! MyPy is happy

In [64]:
trash_can: TrashCan[Refuse] = TrashCan()
deploy(trash_can) # GOOD! MyPy is happy

In [59]:
compost_can: TrashCan[Compostable] = TrashCan()
deploy(compost_can) # incompatible type "TrashCan[Compostable]" expected "TrashCan[Biodegradable]"

In [65]:
# A :> B (A is a supertype of or the same as B)
# B <: A (B is a subtype of or the same as A)
# given A :> B is a generic type C is covariant when C[A] :> C[B]

# Given A :> B, a genderic type K is contravariant if K[A] <: K[B]
# contravariant container is usually a write-only data structure, also known as a sink.

In [66]:
"""
the contravariant formal parameter defines the type of the arguments it is 
used to invoke or send data to the object, different covariant define the types of outputs
produced by the object
"""

'\nthe contravariant formal parameter defines the type of the arguments it is \nused to invoke or send data to the object, different covariant define the types of outputs\nproduced by the object\n'

In [68]:
# covariant - comes out of object
# contravariant - goes into object after its initial contruction
# invariant - comes out of object and dinfes a type for data that goes into object
# Callable[[ParamType, ...], ReturnType] ReturnType is covariant, ParamType is contraviant

# TypeVar creates paramaters that are invariant

In [73]:
import math
from typing import NamedTuple, SupportsAbs

class Vector2d(NamedTuple):
    x: float
    y: float

    def __abs__(self) -> float:
        return math.hypot(self.x, self.y)

def is_unit(v: SupportsAbs[float]) -> bool:
    """True if the magnitude of 'v' is close to 1"""
    return math.isclose(abs(v), 1.0)

assert issubclass(Vector2d, SupportsAbs) # @runtime_checkable in definition of `SupportsAbs`

In [78]:
v0 = Vector2d(0,1)
sqrt2 = math.sqrt(2)
v1 = Vector2d(sqrt2 / 2, sqrt2 / 2)
v2 = Vector2d(1,1)
v3 = complex(.5, math.sqrt(3) / 2)
v4 = 1

assert is_unit(v0)
assert is_unit(v1)
assert not is_unit(v2)
assert is_unit(v3)
assert is_unit(v4)
print('~made it~ all of above assertions: OK!')

~made it~ all of above assertions: OK!


In [79]:
from typing import Protocol, runtime_checkable, TypeVar

T_co = TypeVar('T_co', covariant=True)

@runtime_checkable
class RandomPicker(Protocol[T_co]):
    def pick(self) -> T_co: ...